# Calculating the reserve Factors of the panels 

# Please enter your name 

In [1]:
name = "fabian"

# Imports 

In [2]:
import pandas as pd
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
from formulas.helpers import *
 

# Constants (Changed based on your data)

In [3]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
nu = personal_data[2]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}, nu = {nu}. Please verify.")

#Panel constants 
length = 750
width = 200


Your personal data is: sigma_yield = 490, EModulus = 65420.46, nu = 0.34. Please verify.


# Open stress values & properties of panels 

In [4]:
#Imoport panel stress file and reformat (OLD VERSION)
# paneldf = pd.read_excel(f'../data/{name}/panel.xlsx', index_col=0)
# paneldf = paneldf.drop(['sigmaZZ', 'sigmaXZ', 'sigmaYZ'],axis=1)
# paneldf = paneldf.reset_index()


########
# New version of importing panel stress file
########

paneldf_v2 = pd.read_csv(f'../data/{name}/panel_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')

# Rename the element column to match
#element_component_df.rename(columns={'Element ID': 'element_id'}, inplace=True)
paneldf_v2.rename(columns={'Elements': 'Element ID'}, inplace=True)

# now add the component column to the paneldf_v2 by matching the element_id
paneldf_v2 = add_component_names_to_elements(paneldf_v2, element_component_df)

paneldf_v2.rename(columns={'Loadcase': 'Load Case', 'XX': 'sigmaXX', 'YY': 'sigmaYY', 'XY': 'sigmaXY', 'ZZ': 'sigmaZZ', 'YZ': 'sigmaYZ', 'ZX': 'sigmaXZ'}, inplace=True)
paneldf_v2.drop(columns=['FileID', 'Step', 'Layer', 'sigmaZZ', 'sigmaXZ', 'sigmaYZ'], inplace=True)
paneldf_v2.reset_index(drop=True, inplace=True)
paneldf_v2.set_index('Element ID', inplace=True) # @yannisendr this is possible, since pandas allows non-unique indices, however operations that require an unique index won't work 

# Rename the load case column to match the old one
paneldf_v2['Load Case'] = paneldf_v2['Load Case'].replace({
    1: 'Subcase 1 (LC1)',
    2: 'Subcase 2 (LC2)',
    3: 'Subcase 3 (LC3)',
})

#Import panel properties and reformat
panelPropertiesdf = pd.read_excel(f'../data/{name}/panel_properties.xlsx', index_col=0)
panelPropertiesdf = panelPropertiesdf.reset_index()
panelPropertiesdf = panelPropertiesdf.rename(columns={'elements': 'Element ID'})

paneldf = paneldf_v2

#print(paneldf.head(5))
print(paneldf_v2.head(5))
#print(panelPropertiesdf.head(5))

                  Load Case  sigmaXX  sigmaXY  sigmaYY Component Name
Element ID                                                           
1           Subcase 1 (LC1)   -66.67    26.25    12.75         panel1
2           Subcase 1 (LC1)   -82.97    27.10    20.87         panel1
3           Subcase 1 (LC1)   -67.63    36.97    20.74         panel1
4           Subcase 1 (LC1)   -97.67    34.52     2.42         panel2
5           Subcase 1 (LC1)   -89.50    27.04    25.35         panel2


# Split up the loadcases 

In [5]:
loadCase1df = paneldf[paneldf['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = paneldf[paneldf['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = paneldf[paneldf['Load Case'] == 'Subcase 3 (LC3)']
# Add the data to load case 
loadCase1df = pd.merge(loadCase1df, panelPropertiesdf, on='Element ID')
loadCase2df = pd.merge(loadCase2df, panelPropertiesdf, on='Element ID')
loadCase3df = pd.merge(loadCase3df, panelPropertiesdf, on='Element ID')

#Set index back to Element ID 
loadCase1df = loadCase1df.set_index('Element ID')
loadCase2df = loadCase2df.set_index('Element ID')
loadCase3df = loadCase3df.set_index('Element ID')

#Give some overview 
loadCase1df.head(5)

,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name,mass,thickness
Element ID,,,,,,,
1,Subcase 1 (LC1),-66.67,26.25,12.75,panel1,0.00054,4
2,Subcase 1 (LC1),-82.97,27.10,20.87,panel1,0.00054,4
3,Subcase 1 (LC1),-67.63,36.97,20.74,panel1,0.00054,4
4,Subcase 1 (LC1),-97.67,34.52,2.42,panel2,0.00054,4
5,Subcase 1 (LC1),-89.50,27.04,25.35,panel2,0.00054,4


# Now we need to combine into panels:
-Panel ID (as our ID)
-sigmaXXavg
-sigmaYYavg
-sigmaXYavg
-length
-width
-thickness 


In [6]:
# We can just average the stresses @fabianober Tackle rounding!!!! 
#For load case 1 
panelLC1 = loadCase1df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC1['length'] = [length]*len(panelLC1)
panelLC1['width'] = [width]*len(panelLC1)
panelLC1

#For load case 2 
panelLC2 = loadCase2df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC2['length'] = [length]*len(panelLC2)
panelLC2['width'] = [width]*len(panelLC2)
panelLC2

#For load case 3
panelLC3 = loadCase3df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC3['length'] = [length]*len(panelLC3)
panelLC3['width'] = [width]*len(panelLC3)
panelLC3

panelLC1

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width
Component Name,,,,,,
panel1,-72.423333,18.120000,30.106667,4.0,750,200
panel10,-72.423333,18.120000,30.106667,4.0,750,200
panel2,-94.486667,18.123333,30.006667,4.0,750,200
panel3,-94.623333,18.120000,30.006667,4.0,750,200
panel4,-93.963333,18.120000,30.470000,4.0,750,200
panel5,-93.463333,18.123333,53.250000,4.0,750,200
panel6,-93.463333,18.123333,53.250000,4.0,750,200
panel7,-93.963333,18.120000,30.470000,4.0,750,200
panel8,-94.623333,18.120000,30.006667,4.0,750,200


# Now we can apply the functions to the load cases 

In [7]:
panelLC1[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC1.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC2[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC2.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC3[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC3.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC3

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,,,,
panel1,-36.210000,9.063333,180.626667,4.0,750,200,5.62,5.05,0.49
panel10,-36.210000,9.063333,180.626667,4.0,750,200,5.62,5.05,0.49
panel2,-47.243333,9.060000,180.050000,4.0,750,200,5.62,4.83,0.47
panel3,-47.313333,9.063333,180.053333,4.0,750,200,5.62,4.83,0.47
panel4,-46.983333,9.060000,182.806667,4.0,750,200,5.62,4.84,0.46
panel5,-46.730000,9.060000,319.510000,4.0,750,200,5.62,4.84,0.17
panel6,-46.730000,9.060000,319.510000,4.0,750,200,5.62,4.84,0.17
panel7,-46.983333,9.060000,182.806667,4.0,750,200,5.62,4.84,0.46
panel8,-47.313333,9.063333,180.053333,4.0,750,200,5.62,4.83,0.47


# Combining back into 1 Data Frame to output 

In [8]:
#Drop irrelevant columns
panelLC1 = panelLC1.drop(['length','thickness', 'width'], axis=1)
panelLC2 = panelLC2.drop(['length','thickness', 'width'], axis=1)
panelLC3 = panelLC3.drop(['length','thickness', 'width'], axis=1)
panelLC1.head(3)


,sigmaXX,sigmaYY,sigmaXY,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,
panel1,-72.423333,18.120000,30.106667,5.62,5.05,1.57
panel10,-72.423333,18.120000,30.106667,5.62,5.05,1.57
panel2,-94.486667,18.123333,30.006667,5.62,4.83,1.17


In [9]:
# Rename columns to be unambigous
panelLC1 = panelLC1.rename(columns={"Reserve Factor": "LC 1 RF", 'sigmaXX':'sigmaXXLC1', 'sigmaYY':'sigmaYYLC1', 'sigmaXY':'sigmaXYLC1', 'k_shear':'k_shearLC1', 'k_biaxial':'k_biaxialLC1'})
panelLC2 = panelLC2.rename(columns={"Reserve Factor": "LC 2 RF", 'sigmaXX':'sigmaXXLC2', 'sigmaYY':'sigmaYYLC2', 'sigmaXY':'sigmaXYLC2', 'k_shear':'k_shearLC2', 'k_biaxial':'k_biaxialLC2'})
panelLC3 = panelLC3.rename(columns={"Reserve Factor": "LC 3 RF", 'sigmaXX':'sigmaXXLC3', 'sigmaYY':'sigmaYYLC3', 'sigmaXY':'sigmaXYLC3', 'k_shear':'k_shearLC3', 'k_biaxial':'k_biaxialLC3'})
panelLC1

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF
Component Name,,,,,,
panel1,-72.423333,18.120000,30.106667,5.62,5.05,1.57
panel10,-72.423333,18.120000,30.106667,5.62,5.05,1.57
panel2,-94.486667,18.123333,30.006667,5.62,4.83,1.17
panel3,-94.623333,18.120000,30.006667,5.62,4.83,1.17
panel4,-93.963333,18.120000,30.470000,5.62,4.84,1.18
panel5,-93.463333,18.123333,53.250000,5.62,4.84,1.06
panel6,-93.463333,18.123333,53.250000,5.62,4.84,1.06
panel7,-93.963333,18.120000,30.470000,5.62,4.84,1.18
panel8,-94.623333,18.120000,30.006667,5.62,4.83,1.17


In [10]:
outputDf = pd.concat([panelLC1, panelLC2, panelLC3], axis=1)
outputDf

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF,sigmaXXLC2,sigmaYYLC2,sigmaXYLC2,k_shearLC2,k_biaxialLC2,LC 2 RF,sigmaXXLC3,sigmaYYLC3,sigmaXYLC3,k_shearLC3,k_biaxialLC3,LC 3 RF
Component Name,,,,,,,,,,,,,,,,,,
panel1,-72.423333,18.120000,30.106667,5.62,5.05,1.57,-49.053333,-12.003333,19.936667,5.62,3.04,1.46,-36.210000,9.063333,180.626667,5.62,5.05,0.49
panel10,-72.423333,18.120000,30.106667,5.62,5.05,1.57,-49.053333,-12.003333,19.936667,5.62,3.04,1.46,-36.210000,9.063333,180.626667,5.62,5.05,0.49
panel2,-94.486667,18.123333,30.006667,5.62,4.83,1.17,-62.083333,-12.000000,19.870000,5.62,3.23,1.24,-47.243333,9.060000,180.050000,5.62,4.83,0.47
panel3,-94.623333,18.120000,30.006667,5.62,4.83,1.17,-63.016667,-12.000000,19.873333,5.62,3.24,1.22,-47.313333,9.063333,180.053333,5.62,4.83,0.47
panel4,-93.963333,18.120000,30.470000,5.62,4.84,1.18,-62.893333,-12.003333,20.180000,5.62,3.24,1.22,-46.983333,9.060000,182.806667,5.62,4.84,0.46
panel5,-93.463333,18.123333,53.250000,5.62,4.84,1.06,-62.590000,-12.000000,35.266667,5.62,3.23,1.16,-46.730000,9.060000,319.510000,5.62,4.84,0.17
panel6,-93.463333,18.123333,53.250000,5.62,4.84,1.06,-62.590000,-12.000000,35.266667,5.62,3.23,1.16,-46.730000,9.060000,319.510000,5.62,4.84,0.17
panel7,-93.963333,18.120000,30.470000,5.62,4.84,1.18,-62.893333,-12.003333,20.180000,5.62,3.24,1.22,-46.983333,9.060000,182.806667,5.62,4.84,0.46
panel8,-94.623333,18.120000,30.006667,5.62,4.83,1.17,-63.016667,-12.000000,19.873333,5.62,3.24,1.22,-47.313333,9.063333,180.053333,5.62,4.83,0.47


# Output the file 

In [11]:
outputDf.to_excel(f'../data/{name}/output/processed_e.xlsx')